In [4]:
#import stuff 
from wordle import Wordle
from collections import Counter
import math
try: 
    from tqdm import tqdm
except ImportError as e :
    tqdm = lambda x: x


In [5]:
# new way of inputting information 

# Here to input information, put in your five letter guess, 
# followed by the response in the following way: 
#  put ('guess','response'), on the list, where each character of 'response' 
#  indicates the color that the corresponding character of the guess turned: 
#    *  if the Tile turned green,  put 'g'
#    *  if the Tile turned yellow, put 'y'
#    *  if the Tile turned gray,   put 'x' 
# so for example if you guessed 'tears', and the correct word was 'rebut', 
#       the response string would be 'ygxyx'

# You should always start with the guess 'tears' or 'soare' or 'trace', rather than re-running this program 
wordle = Wordle() # create instance of wordle words object

Guesses = [
    # ('soare','gxxxx'),  
    # ('thilk','xxggy'),
    # ('sight','gyxxx'),
    # ('slick','gygxy'),
#     ('joint','xgggg')
]

In [16]:
known_letters = {}
yellow_letters = []
wrong_letters = ''
for guess,resp in Guesses: 
    for i in range(5): 
        if resp[i] == 'g': 
            known_letters[i] = guess[i]
        elif resp[i] == 'y': 
            yellow_letters.append((i,guess[i]))
        else: 
            wrong_letters += guess[i]
correct_lets = [let for _,let in yellow_letters] + list(known_letters.values())   
wrong_letters = ''.join(c for c in wrong_letters if c not in correct_lets)
print(known_letters)
print(yellow_letters)
print(wrong_letters)

{}
[]



In [21]:
possible_words = wordle.sc_words

# start by narrowing based off of known letters 
for ind, val in known_letters.items():
    if val == '' : 
        continue 
    print(f"narrowing based off of '{val}' in spot {ind}")
    possible_words = [word for word in  possible_words if word[ind] == val]
    print(f"\t list size is {len(list(possible_words))}")

# then look at all yellow letters 
for ind, val in yellow_letters:
    if val == '' : 
        continue 
    print(f"narrowing based off of '{val}' in spot {ind}")
    possible_words = [word for word in  possible_words if val in word and word[ind] != val]
    print(f"\t list size is {len(list(possible_words))}")
# finally look at all letters not in the words 

print("narrowing words based off of incorrect letters")
for let in wrong_letters: 
    print(f"narrowing based off of incorrect letter '{let}'")
#     print(let not in known_letters.values())
#     print(list(known_letters.values()))
    possible_words = [word for word in  possible_words if let not in word or let in known_letters.values()]
#     print(f'list is {possible_words}')
print(f"\t list size is {len(list(possible_words))}")

if len(list(possible_words))  < 40: 
    print(f"all possible words are: {list(possible_words)}")
else: 
    print('Too many words to display') 

narrowing words based off of incorrect letters
	 list size is 2315
Too many words to display


In [22]:
# hard mode based geussing
hard_mode = 'Default' # currently hard mode is not garunteed to be 'optimal'. 
                 # But when only a few options are avaliable, it often performs better.
use_count = False;

if hard_mode == 'Default': 
    hard_mode  = len(possible_words) <= 8
    print(f"using hard mode: {hard_mode}")
    
def calc_shannon_entropy(counts:Counter):
    all_counts = [count for _,count in counts.items()]
    all_words = sum(all_counts)
    probs = (count / all_words  for count in all_counts)
    return sum((-1*prob *math.log(prob,2) for prob in probs))

    
        

possible_guesses = possible_words  if hard_mode else wordle.words
best_metric = 0 
best_guesses = []
for guess in tqdm(possible_guesses):
    resps = []
    for ans in possible_words: 
        resp = ['g' if ans[i] == c else 'y' if c in ans else 'x' for i,c in enumerate(guess)]
        resps.append(''.join(resp))
    
    resp_counts = Counter(resps)
    num_uniques = len(resp_counts)
    entropy = calc_shannon_entropy(resp_counts)
    metric = num_uniques if use_count else entropy
#     print(f"{guess} has resp_count: {num_uniques} and entropy = {calc_shannon_entropy(resp_counts)} ")

    if metric > best_metric: 
        best_guesses = []
        best_metric = metric
        
    if metric >= best_metric: 
        entr = {
            "guess":        guess,
            "num_uniques":  num_uniques, 
            "resp_distro":  resp_counts,
            "entropy bits": entropy
        }
        best_guesses.append(entr)
    
for guess in best_guesses:
    printafterline=False
    if guess['guess'] in possible_words and not hard_mode:
        printafterline = True 
        print("Reccomended word: ")
    print(guess)
    if printafterline : 
        print(" ")
    

using hard mode: False


100%|████████████████████████████████████| 12972/12972 [00:43<00:00, 298.55it/s]

{'guess': 'soare', 'num_uniques': 127, 'resp_distro': Counter({'xxxxx': 183, 'xxyxx': 138, 'xxxxy': 120, 'xxxyy': 117, 'xgxxx': 87, 'xxxxg': 79, 'xxyxy': 68, 'gxxxx': 63, 'xxyyy': 61, 'xyxyx': 59, 'xxxyx': 57, 'xyxxx': 56, 'xyyxx': 44, 'xxyyx': 42, 'gxgxx': 41, 'xxgxx': 40, 'xxyxg': 40, 'xxgyx': 40, 'xxxyg': 39, 'gyxxx': 37, 'gxxxy': 35, 'yxxxx': 33, 'xxgxg': 28, 'xyyyx': 27, 'xgxxy': 26, 'xgxyx': 26, 'xyxxy': 25, 'yxyxx': 23, 'gxxxg': 23, 'xgxyy': 22, 'xxgxy': 21, 'xgyxx': 21, 'xyxxg': 20, 'yxxxy': 19, 'xxgyg': 19, 'gxyxx': 19, 'gxgxg': 19, 'yxxyx': 16, 'yxyxg': 15, 'ygxxx': 15, 'yxxyy': 15, 'yxgxx': 14, 'xxxgy': 14, 'xyxyg': 14, 'xxggx': 13, 'xxxgx': 13, 'xyxyy': 13, 'gxxyy': 13, 'gyxxg': 12, 'xxygx': 11, 'ygxxg': 11, 'gxggx': 11, 'xgxxg': 10, 'gyxgx': 10, 'xxggy': 9, 'xgxyg': 9, 'gxxyx': 9, 'gxyxy': 9, 'xyyxg': 8, 'xgyyx': 8, 'xyxgx': 8, 'yxxxg': 8, 'gxyyx': 8, 'gxxgx': 8, 'yxgxg': 7, 'xxyyg': 7, 'yyxxx': 7, 'yxgyx': 7, 'yxxyg': 7, 'ggxxx': 7, 'xyygx': 6, 'xgxgx': 6, 'gxyyy': 6, 'gy

In [1]:
%load_ext autoreload
%autoreload 2
from wordle import Wordle 



In [36]:
wordle = Wordle()
Guesses = [
    ('soare','gxxxx'),  
    ('sight','gyxxx'),
    # ('slick','gygxy'),
#     ('joint','xgggg')
]
wordle.add_guesses(Guesses, reset_info=True) 

In [37]:
valid_words = wordle.get_possible_words()

print(f'Found {len(valid_words)} valid_words')
if len(valid_words) < 50: 
    print(valid_words)
else: 
    print("Too many words to display")

Found 13 valid_words
['skiff', 'skill', 'skimp', 'slick', 'slimy', 'slink', 'sniff', 'spicy', 'spiky', 'spill', 'spiny', 'squib', 'swill']


In [44]:
opt_guesses = wordle.get_best_guesses(use_entropy = True, return_arr = True,use_tqdm= False, give_distribution=True)
opt_guesses

using hard mode: False
best guess found gave 2.458311329683084 bits of information


[{'guess': 'plank', 'num_classes': 12, 'entropy bits': 2.458311329683084},
 {'guess': 'plink', 'num_classes': 12, 'entropy bits': 2.458311329683084},
 {'guess': 'plonk', 'num_classes': 12, 'entropy bits': 2.458311329683084},
 {'guess': 'pluck', 'num_classes': 12, 'entropy bits': 2.458311329683084},
 {'guess': 'plunk', 'num_classes': 12, 'entropy bits': 2.458311329683084}]